# Transforming

In this notebook, we will explore how to use Large Language Models for text transformation tasks such as language translation, spelling and grammar checking, tone adjustment, and format conversion.

## Setup

In [1]:
import openai
import os

print ("OpenAI version:", openai.__version__)
OPENAI_KEY = os.getenv('OPENAI_API_KEY')

OpenAI version: 1.30.5


In [2]:
# client = openai.OpenAI(api_key=OPENAI_KEY)
client = openai.OpenAI(
    base_url = 'http://localhost:11434/v1',
    api_key='ollama', # required, but unused
)

def get_completion(prompt, model="llama3"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

## Translation

ChatGPT is trained with sources in many languages. This gives the model the ability to do translation. Here are some examples of how to use this capability.

In [17]:
prompt = f"""
Translate the following English text to Spanish: \ 
```Hi, I would like to order a salad```

Do not output anything else.
"""
response = get_completion(prompt)
print(response)

Hola, me gustaría ordenar una ensalada.


In [16]:
prompt = f"""
Tell me which language this is: 
```Combien coûte le lampadaire?```

Do not output anything else.
"""
response = get_completion(prompt)
print(response)

French


In [18]:
prompt = f"""
Translate the following  text to French and Spanish
and English pirate: \
```I want to order a basketball```

Do not output anything else.
"""
response = get_completion(prompt)
print(response)

French:
Je veux commander un panier à basket.

Spanish:
Quiero ordenar una canasta de baloncesto.

English (Pirate):
Arr, I be wantin' to order yerself a hoop-ball basket!


In [20]:
prompt = f"""
Translate the following text to Spanish in both the \
formal and informal forms: 
'Would you like to order a pillow?'

Do not output anything else.
"""
response = get_completion(prompt)
print(response)

Formal:
¿Le gustaría ordenar un almohadón?

Informal:
¿Quieres pedir un almohadón?


### Universal Translator
Imagine you are in charge of IT at a large multinational e-commerce company. Users are messaging you with IT issues in all their native languages. Your staff is from all over the world and speaks only their native languages. You need a universal translator!

In [7]:
user_messages = [
  "La performance du système est plus lente que d'habitude.",  # System performance is slower than normal         
  "Mi monitor tiene píxeles que no se iluminan.",              # My monitor has pixels that are not lighting
  "Il mio mouse non funziona",                                 # My mouse is not working
  "Mój klawisz Ctrl jest zepsuty",                             # My keyboard has a broken control key
  "我的屏幕在闪烁"                                               # My screen is flashing
] 

In [8]:
for issue in user_messages:
    prompt = f"Tell me what language this is: ```{issue}```"
    lang = get_completion(prompt)
    print(f"Original message ({lang}): {issue}")

    prompt = f"""
    Translate the following  text to English \
    and Korean: ```{issue}```
    """
    response = get_completion(prompt)
    print(response, "\n")

Original message (This text is in French.): La performance du système est plus lente que d'habitude.
English translation:

"The system's performance is slower than usual."

Korean translation:



(Li-si-teu-eui pyeong-seong-i kka-ma-yeo-yo) 

Original message (A Spanish sentence!

Here's a breakdown:

* "Mi" means "my"
* "monitor" means "monitor" (as in, a computer monitor)
* "tiene" is the verb "to have" in the third person singular present tense
* "píxeles" means "pixels"
* "que" is a conjunction meaning "that" or "which"
* "no se iluminan" is the phrase "do not illuminate themselves", where:
	+ "no" means "not"
	+ "se" is the reflexive pronoun "oneself"
	+ "iluminan" is the verb "to illuminate" in the third person plural present tense

So, the sentence translates to: "My monitor has pixels that don't light up."): Mi monitor tiene píxeles que no se iluminan.
English translation:

"My monitor has pixels that don't light up."

Korean translation:



(Myeo moniteo-neun pikeuseu ga neomu

## Tone Transformation
Writing can vary based on the intended audience. ChatGPT can produce different tones.


In [9]:
prompt = f"""
Translate the following from slang to a business letter: 
'Dude, This is Joe, check out this spec on this standing lamp.'
"""
response = get_completion(prompt)
print(response)

Here's the translation:

Dear [Recipient],

I wanted to bring to your attention an exceptional opportunity regarding a standing lamp. I am excited to share the details with you.

Sincerely,
Joe


## Format Conversion
ChatGPT can translate between formats. The prompt should describe the input and output formats.

In [10]:
data_json = { "resturant employees" :[ 
    {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
    {"name":"Bob", "email":"bob32@gmail.com"},
    {"name":"Jai", "email":"jai87@gmail.com"}
]}

prompt = f"""
Translate the following python dictionary from JSON to an HTML \
table with column headers and title: {data_json}
"""
response = get_completion(prompt)
print(response)

Here is the Python code to translate the JSON dictionary into an HTML table with column headers and title:
```
import json
from tabulate import tabulate

# Load the JSON data
data = json.loads('{"resturant employees": [{"name": "Shyam", "email": "shyamjaiswal@gmail.com"}, {"name": "Bob", "email": "bob32@gmail.com"}, {"name": "Jai", "email": "jai87@gmail.com"}]}')

# Extract the data and headers
headers = ['Name', 'Email']
table_data = [[row['name'], row['email']] for row in data['resturant employees']]

# Create the HTML table
html_table = tabulate(table_data, headers=headers, tablefmt='html')

# Print the HTML table with title
print('<h1>Resturant Employees</h1>')
print(html_table)
```
This code uses the `json` module to load the JSON data, and the `tabulate` library to create the HTML table. The `headers` variable defines the column headers, and the `table_data` list comprehension extracts the data from the JSON dictionary.

The resulting HTML table will look like this:
```
<h1>Restu

In [11]:
from IPython.display import display, Markdown, Latex, HTML, JSON
display(HTML(response))

Name,Email
Shyam,shyamjaiswal@gmail.com
Bob,bob32@gmail.com
Jai,jai87@gmail.com


## Spellcheck/Grammar check.

Here are some examples of common grammar and spelling problems and the LLM's response. 

To signal to the LLM that you want it to proofread your text, you instruct the model to 'proofread' or 'proofread and correct'.

In [23]:
text = [ 
  "The girl with the black and white puppies have a ball.",  # The girl has a ball.
  "Yolanda has her notebook.", # ok
  "Its going to be a long day. Does the car need it’s oil changed?",  # Homonyms
  "Their goes my freedom. There going to bring they’re suitcases.",  # Homonyms
  "Your going to need you’re notebook.",  # Homonyms
  "That medicine effects my ability to sleep. Have you heard of the butterfly affect?", # Homonyms
  "This phrase is to cherck chatGPT for speling abilitty"  # spelling
]
for t in text:
    prompt = f"""Proofread and correct the following text
    and rewrite the corrected version. If you don't find
    and errors, just say "No errors found". 
    Do not output anything else and don't explain the corrections. Don't use 
    any punctuation around the text:
    ```{t}```"""
    response = get_completion(prompt)
    print(response)

Here is the corrected version:

```
The girl with the black and white puppies has a ball.
```
Here is the corrected text:

Yolanda has her notebook.
```
It's going to be a long day. Does the car need its oil changed?
```
```
There goes my freedom. They're going to bring their suitcases.
```
```
You're going to need your notebook.
```
Here is the corrected version:

```
That medicine affects my ability to sleep. Have you heard of the butterfly effect?
```
Here is the corrected version:

```
This phrase is to check ChatGPT's spelling ability.
```


In [26]:
text = f"""
Got this for my daughter for her birthday cuz she keeps taking \
mine from my room.  Yes, adults also like pandas too.  She takes \
it everywhere with her, and it's super soft and cute.  One of the \
ears is a bit lower than the other, and I don't think that was \
designed to be asymmetrical. It's a bit small for what I paid for it \
though. I think there might be other options that are bigger for \
the same price.  It arrived a day earlier than expected, so I got \
to play with it myself before I gave it to my daughter.
"""
prompt = f"proofread and correct this review: ```{text}```. Output only the corrected version. Do not output anything else."
response = get_completion(prompt)
print(response)

Here is the corrected review:

I purchased this panda plush toy for my daughter's birthday because she keeps taking mine from my room. Yes, adults also appreciate pandas! She takes it everywhere with her, and it's incredibly soft and adorable. One of the ears appears to be slightly lower than the other, which I don't think was intentionally designed as asymmetrical. While it's a cute toy, I do think it's a bit small for what I paid for it. I'm not convinced that there aren't larger options available at the same price point. The item arrived a day earlier than expected, so I got to enjoy it myself before giving it to my daughter.


In [28]:
from redlines import Redlines

diff = Redlines(text,response)
display(Markdown(diff.output_markdown))

<span style='color:red;font-weight:700;text-decoration:line-through;'>Got </span><span style='color:green;font-weight:700;'>Here is the corrected review: </span>

<span style='color:green;font-weight:700;'>I purchased </span>this <span style='color:green;font-weight:700;'>panda plush toy </span>for my <span style='color:red;font-weight:700;text-decoration:line-through;'>daughter for her </span><span style='color:green;font-weight:700;'>daughter's </span>birthday <span style='color:red;font-weight:700;text-decoration:line-through;'>cuz </span><span style='color:green;font-weight:700;'>because </span>she keeps taking mine from my <span style='color:red;font-weight:700;text-decoration:line-through;'>room.  </span><span style='color:green;font-weight:700;'>room. </span>Yes, adults also <span style='color:red;font-weight:700;text-decoration:line-through;'>like pandas too.  </span><span style='color:green;font-weight:700;'>appreciate pandas! </span>She takes it everywhere with her, and it's <span style='color:red;font-weight:700;text-decoration:line-through;'>super </span><span style='color:green;font-weight:700;'>incredibly </span>soft and <span style='color:red;font-weight:700;text-decoration:line-through;'>cute.  </span><span style='color:green;font-weight:700;'>adorable. </span>One of the ears <span style='color:red;font-weight:700;text-decoration:line-through;'>is a bit </span><span style='color:green;font-weight:700;'>appears to be slightly </span>lower than the other, <span style='color:red;font-weight:700;text-decoration:line-through;'>and </span><span style='color:green;font-weight:700;'>which </span>I don't think <span style='color:red;font-weight:700;text-decoration:line-through;'>that </span>was <span style='color:green;font-weight:700;'>intentionally </span>designed <span style='color:red;font-weight:700;text-decoration:line-through;'>to be </span><span style='color:green;font-weight:700;'>as </span>asymmetrical. <span style='color:red;font-weight:700;text-decoration:line-through;'>It's </span><span style='color:green;font-weight:700;'>While it's a cute toy, I do think it's </span>a bit small for what I paid for <span style='color:red;font-weight:700;text-decoration:line-through;'>it though. I think </span><span style='color:green;font-weight:700;'>it. I'm not convinced that </span>there <span style='color:red;font-weight:700;text-decoration:line-through;'>might be other </span><span style='color:green;font-weight:700;'>aren't larger </span>options <span style='color:red;font-weight:700;text-decoration:line-through;'>that are bigger for </span><span style='color:green;font-weight:700;'>available at </span>the same <span style='color:red;font-weight:700;text-decoration:line-through;'>price.  It </span><span style='color:green;font-weight:700;'>price point. The item </span>arrived a day earlier than expected, so I got to <span style='color:red;font-weight:700;text-decoration:line-through;'>play with </span><span style='color:green;font-weight:700;'>enjoy </span>it myself before <span style='color:red;font-weight:700;text-decoration:line-through;'>I gave </span><span style='color:green;font-weight:700;'>giving </span>it to my daughter.

In [29]:
prompt = f"""
proofread and correct this review. Make it more compelling. 
Ensure it follows APA style guide and targets an advanced reader. 
Output in markdown format.
Text: ```{text}```
"""
response = get_completion(prompt)
display(Markdown(response))

Here is the proofread and rewritten review in markdown format:

**A Soft and Cuddly Companion**
================================

I purchased this plush panda for my daughter's birthday, as she had been borrowing mine from my room. While adults may also appreciate these adorable creatures, I was pleased to see that it brought a smile to her face. The softness and cuteness of the toy are undeniable.

Upon closer inspection, I noticed that one ear is slightly lower than the other, which may not have been intentionally designed as asymmetrical. Nevertheless, this minor flaw did not detract from my daughter's enthusiasm for the plush toy.

While the size of the panda was satisfactory, I felt that it was a tad smaller than expected considering the price point. In hindsight, I might have explored alternative options that offer greater value for the same cost.

A pleasant surprise was the timely arrival of the package, which arrived a day earlier than anticipated. This allowed me to enjoy the plush toy myself before presenting it to my daughter, making the experience even more special.

**APA Style Guide Compliance:**

* The review is written in the first person (I) and uses present tense verbs.
* The text includes specific details about the product, such as its softness and cuteness.
* The reviewer provides a balanced assessment of the product's pros and cons.
* The review is concise and easy to read.

**Target Audience:**

* Advanced readers who are interested in reading reviews that provide detailed information about products.
* Individuals who value honesty and transparency in their purchasing decisions.

## Try it yourself!
Try changing the instructions to form your own review.

Thanks to the following sites:

https://writingprompts.com/bad-grammar-examples/
